In [1]:
import os
import json

from datasets import Dataset
from pathlib import Path
# Paths
training_folder = Path("dataset/training")  
programs_path = training_folder/"Programs"

invariants_path = training_folder / "invariants.json"
print(f"invariants_path: {invariants_path}\nprograms_path: {programs_path}")
LIMIT = 5

invariants_path: dataset/training/invariants.json
programs_path: dataset/training/Programs


In [2]:
# Load invariants.json
with open(invariants_path, 'r') as f:
    invariants_data = json.load(f)
    
print(f"number of files: {len(invariants_data)}")

number of files: 3589


In [ ]:
from typing import Any


c_filenames = list[Any](invariants_data.keys())[:LIMIT]
print(f"number of C filenames: {len(c_filenames)}")


number of C filenames: 5


In [5]:
from src.utils.rewriter import Rewriter


In [6]:
example_filename = c_filenames[0]
full_c_path = programs_path / example_filename
print(f"example_filename: {example_filename}")
r = Rewriter(filename=full_c_path, rewrite=False)
print(r)

example_filename: 1003_1.c


In [9]:
# dict_keys(['code',
# 'new_code',
# 'has_reach_error', 
# 'lines_to_verify',
# 'lines_for_gpt', 
# 'replacement'])
# for k,v in r.__dict__.items():
#     print(f"{k}: {v}")
for i in range(len(r.lines_to_verify)):
    print(r.lines_to_verify[i])

#include <assert.h>
void reach_error(void) { assert(0); }

extern int __VERIFIER_nondet_int(void);
extern _Bool __VERIFIER_nondet_bool(void);

void __VERIFIER_assert(int cond) {
    if (!cond) {
        reach_error();
    }
}

/* Custom CFG:
names=i count isPositive
beforeloop=
beforeloopinit=
precondition=i==0 && count==0 && isPositive==1
loopcondition=i<200 && isPositive
loop=count += i; i += 2; isPositive = (i % 5 != 0);
postcondition=count >= 0
afterloop=
learners= conj
*/
int main() {
    int i = __VERIFIER_nondet_int();
    int count = __VERIFIER_nondet_int();
    _Bool isPositive = __VERIFIER_nondet_bool();

    if (!(i == 0 && count == 0 && isPositive == 1)) {
        return 0;
    }

    while (i < 200 && isPositive) {
        count += i;
        i += 2;
        isPositive = (i % 5 != 0);
    }

    __VERIFIER_assert(count >= 0);
    return 0;
}


In [10]:
for i in range(len(r.lines_for_gpt)):
    print(r.lines_for_gpt[i])


#include <assert.h>
void reach_error(void) { assert(0); }

extern int __VERIFIER_nondet_int(void);
extern _Bool __VERIFIER_nondet_bool(void);

void __VERIFIER_assert(int cond) {
    if (!cond) {
        reach_error();
    }
}

/* Custom CFG:
names=i count isPositive
beforeloop=
beforeloopinit=
precondition=i==0 && count==0 && isPositive==1
loopcondition=i<200 && isPositive
loop=count += i; i += 2; isPositive = (i % 5 != 0);
postcondition=count >= 0
afterloop=
learners= conj
*/
int main() {
    int i = __VERIFIER_nondet_int();
    int count = __VERIFIER_nondet_int();
    _Bool isPositive = __VERIFIER_nondet_bool();

    if (!(i == 0 && count == 0 && isPositive == 1)) {
        return 0;
    }

    while (i < 200 && isPositive) {
        count += i;
        i += 2;
        isPositive = (i % 5 != 0);
    }

    __VERIFIER_assert(count >= 0);
    return 0;
}


In [16]:
from src.utils.program import Program
program = Program(r.lines_to_verify, r.replacement)
print(program)

#include <assert.h>
void reach_error(void) { assert(0); }

extern int __VERIFIER_nondet_int(void);
extern _Bool __VERIFIER_nondet_bool(void);

void __VERIFIER_assert(int cond) {
    if (!cond) {
        reach_error();
    }
}

/* Custom CFG:
names=i count isPositive
beforeloop=
beforeloopinit=
precondition=i==0 && count==0 && isPositive==1
loopcondition=i<200 && isPositive
loop=count += i; i += 2; isPositive = (i % 5 != 0);
postcondition=count >= 0
afterloop=
learners= conj
*/
int main() {
    int i = __VERIFIER_nondet_int();
    int count = __VERIFIER_nondet_int();
    _Bool isPositive = __VERIFIER_nondet_bool();

    if (!(i == 0 && count == 0 && isPositive == 1)) {
        return 0;
    }

    while (i < 200 && isPositive) {
        count += i;
        i += 2;
        isPositive = (i % 5 != 0);
    }

    __VERIFIER_assert(count >= 0);
    return 0;
}
--------------------------------- Program ---------------------------------------

📝 Program lines without assertions (41 total):
---

In [14]:
print(invariants_data[example_filename])

[{'line': 33, 'invariant': '0 <= count && 0 <= i'}]


In [ ]:


# Get first 5 file names

# Build list of dicts for each file: {'code', 'invariant', 'line'}
data  = []
for filename in invariants_data.keys():
    # Try to read code
    program_path = programs_path / filename
    try:
        with open(program_path, 'r') as code_file:
            code = code_file.read()
            r = Rewriter(filename=program_path, rewrite=False)
            p = Program(r.lines_to_verify, r.replacement)
            code = "\n".join(p.lines)
    except FileNotFoundError:
        code = None  # or handle otherwise

    # Get invariants for this file (may be a list)
    invariants = invariants_data[filename]
    # For each invariant/line pair, make a dict, but include the code each time
    for inv in invariants:
        data.append({
            'code': code,
            'invariant': inv.get('invariant'),
            'line': inv.get('line'),
            'file': filename
        })

# Now, first_5_data is a list of dicts, one per (filename, invariant/line).


In [ ]:
developer_msg = "You are a helpful assistant and an expert C programmer."
user_msg = "Generate a strong loop invariant that helps prove the target property of the following C program: \n```c\n{program}\n```\n\nAvailable locations for placing the invariant:\n{locations}\n\nOutput Format:\nassert(<invariant>); // Line <line_number>"
response_msg = "assert({invariant}); // Line {line_number}"
samples = []
for example in data:
    sample = {
        "messages": [
            {"role": "system", "content": developer_msg},
            {"role": "user", "content": user_msg.format(program=example['code'], locations=example['line'])},
            {"role": "assistant", "content": response_msg.format(invariant=example['invariant'], line_number=example['line'])}
        ]
    }
    samples.append(sample)
print(samples[0])